In [5]:
import os
from __future__ import print_function

master = '--master local[1]'
#master = '--master spark://apachespark-master-2-1-0:7077'
jars = '--jars /root/lib/jpmml-sparkml-package-1.2-SNAPSHOT.jar'
#packages = '--packages org.jpmml:jpmml-sparkml:1.2.0'
packages = ''
py_files = '--py-files /root/lib/jpmml.py'

os.environ['PYSPARK_SUBMIT_ARGS'] = master \
  + ' ' + jars \
  + ' ' + packages \
  + ' ' + py_files \
  + ' ' + 'pyspark-shell'

print(os.environ['PYSPARK_SUBMIT_ARGS'])

--master local[1] --jars /root/lib/jpmml-sparkml-package-1.2-SNAPSHOT.jar  --py-files /root/lib/jpmml.py pyspark-shell


In [7]:
from pyspark.ml import Pipeline

from pyspark import SparkConf, SparkContext
from pyspark.sql import SQLContext
from pyspark.ml.feature import VectorAssembler


In [8]:
sc = SparkContext.getOrCreate()
sqlContext = SQLContext(sc)

In [9]:
import pandas as pd
from sklearn import datasets
iris = datasets.load_iris()

df = pd.DataFrame(iris.data, columns=iris.feature_names)

df['species'] = pd.Categorical.from_codes(iris.target, iris.target_names)
df['species'] = pd.factorize(df['species'])[0]

df.columns = ["sepal_length(cm)", \
              "sepal_width(cm)", \
              "petal_length(cm)", \
              "petal_width(cm)", \
              "species"]

df_iris = sqlContext.createDataFrame(df)

df_iris.printSchema()

df_iris.show(5)

train = df_iris

#train, test = dfiris.randomSplit([0.7, 0.3])

#train.count()
#test.count()

root
 |-- sepal_length(cm): double (nullable = true)
 |-- sepal_width(cm): double (nullable = true)
 |-- petal_length(cm): double (nullable = true)
 |-- petal_width(cm): double (nullable = true)
 |-- species: long (nullable = true)

+----------------+---------------+----------------+---------------+-------+
|sepal_length(cm)|sepal_width(cm)|petal_length(cm)|petal_width(cm)|species|
+----------------+---------------+----------------+---------------+-------+
|             5.1|            3.5|             1.4|            0.2|      0|
|             4.9|            3.0|             1.4|            0.2|      0|
|             4.7|            3.2|             1.3|            0.2|      0|
|             4.6|            3.1|             1.5|            0.2|      0|
|             5.0|            3.6|             1.4|            0.2|      0|
+----------------+---------------+----------------+---------------+-------+
only showing top 5 rows



In [26]:
from pyspark.ml.classification import LogisticRegression

from pyspark.ml import Pipeline

assembler = VectorAssembler(inputCols=["sepal_length(cm)", \
                                       "sepal_width(cm)", \
                                       "petal_length(cm)", \
                                       "petal_width(cm)"], 
                            outputCol="features")

lr = LogisticRegression(featuresCol='features', labelCol='species')

pipeline = Pipeline(stages=[assembler, lr])

pipeline_model = pipeline.fit(train)

print(pipeline_model)

PipelineModel_48dc9821a575394c8e73


In [30]:
lr_model = pipeline_model.stages[1]
print(lr_model.numClasses)

3


In [31]:
train.schema.

StructType(List(StructField(sepal_length(cm),DoubleType,true),StructField(sepal_width(cm),DoubleType,true),StructField(petal_length(cm),DoubleType,true),StructField(petal_width(cm),DoubleType,true),StructField(species,LongType,true)))

In [28]:
prediction_lr = pipeline_model.transform(test)

score_lr = prediction_lr.select(['species', 'prediction'])
score_lr.show(n=score_lr.count())

prediction_lr.show(5)

+----------+----------+
|   species|prediction|
+----------+----------+
|    setosa|       0.0|
|    setosa|       0.0|
|    setosa|       0.0|
|    setosa|       0.0|
|    setosa|       0.0|
|    setosa|       0.0|
|    setosa|       0.0|
|versicolor|       1.0|
| virginica|       2.0|
|    setosa|       0.0|
|versicolor|       1.0|
|versicolor|       1.0|
|    setosa|       0.0|
|    setosa|       0.0|
|    setosa|       0.0|
|    setosa|       0.0|
|    setosa|       0.0|
|    setosa|       0.0|
|    setosa|       0.0|
|    setosa|       0.0|
|    setosa|       0.0|
|    setosa|       0.0|
|versicolor|       1.0|
| virginica|       2.0|
| virginica|       2.0|
|versicolor|       1.0|
|    setosa|       0.0|
|versicolor|       1.0|
|versicolor|       1.0|
| virginica|       2.0|
| virginica|       2.0|
|    setosa|       0.0|
|versicolor|       1.0|
|versicolor|       1.0|
| virginica|       2.0|
| virginica|       2.0|
|versicolor|       1.0|
|versicolor|       1.0|
| virginica|    

In [6]:
from jpmml import toPMMLBytes

pmmlBytes = toPMMLBytes(sc, train, lr_model)
print(pmmlBytes.decode("UTF-8"))

ImportError: No module named 'jpmml'